In [1]:
# !pip install pulp
# !pip install tqdm

In [2]:
import numpy as np
import pandas as pd

In [3]:
data_path='/수요지_행정동_분류.csv'
data=pd.read_csv(data_path)
def extract_dong(full_name):
    return full_name.split()[-1]

# 새로운 '동명' 컬럼 생성
data['동명'] = data['행정동명'].apply(extract_dong)
data.drop('행정동명', axis=1, inplace=True)
data['동명'] = data['동명'].str.replace('\d', '').str.replace('·', '')

In [4]:
target_path='/target_행정동_분류.csv'
target=pd.read_csv(target_path)

# NaN 값을 갖는 행 필터링
nan_rows = target[target['행정동명'].isna()]

# 해당 행의 '학교 id' 출력
print(nan_rows['학교 id'].tolist())

In [5]:
target=target.dropna()
target

In [6]:
# 새로운 '동명' 컬럼 생성
target['동명'] = target['행정동명'].apply(extract_dong)
target.drop('행정동명', axis=1, inplace=True)
target['동명'] = target['동명'].str.replace('\d', '').str.replace('·', '')
target

true
true
true
true
true
true
인천광역시 중구 송월동
true
true
true
true
true
true
인천광역시 중구 율목동
인천광역시 동구 송현1·2동
true
인천광역시 동구 송림1동
true
인천광역시 동구 송현3동
true
true
true
인천광역시 동구 송림4동


In [7]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (킬로미터)

    # 위도와 경도를 라디안으로 변환
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # 위도와 경도의 차이 계산
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # 허버사인 공식
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distance = R * c

    return distance

In [8]:
s=data['초등학령인구'].sum()
s/750

In [9]:
s/200

In [10]:
import geopandas as gpd

# 인천 읍면동 경계 파일 읽기
incheon_boundary = gpd.read_file('/content/drive/MyDrive/행정동.geojson', encoding='cp949')

incheon_boundary = gpd.read_file('/content/drive/MyDrive/행정동.geojson', encoding='utf-8')

In [11]:
# 데이터 프레임을 geopandas 데이터 프레임으로 변환
gdf_target = gpd.GeoDataFrame(target, geometry=gpd.points_from_xy(target.X, target.Y))
gdf_data = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data.X, data.Y))

# 좌표계 일치화
gdf_target = gdf_target.set_crs(incheon_boundary.crs)
gdf_data = gdf_data.set_crs(incheon_boundary.crs)

# 3. target과 data의 각 좌표가 어느 읍면동에 속하는지 판별
def get_dong_name(gdf, boundaries):
    for _, boundary in boundaries.iterrows():
        mask = gdf.within(boundary['geometry'])
        if mask.any():
            gdf.loc[mask, 'name_eng'] = boundary['name_eng']
    return gdf

gdf_target = get_dong_name(gdf_target, incheon_boundary)
gdf_data = get_dong_name(gdf_data, incheon_boundary)

# 4. 결과를 원래의 데이터 프레임에 추가
target['name_eng'] = gdf_target['name_eng']
data['name_eng'] = gdf_data['name_eng']

In [12]:
gdf_target

In [ ]:
# 인접한 동의 정보를 저장할 딕셔너리
adjacent_dongs = {}

# 각 동에 대해 인접한 동을 찾음
for idx, row in incheon_boundary.iterrows():
    dong_name = row['name']
    # 해당 동의 경계를 가지고 옴
    boundary = row['geometry']
    # 인접한 동을 찾음
    neighbors = incheon_boundary[incheon_boundary['geometry'].touches(boundary)]['name'].tolist()
    adjacent_dongs[dong_name] = neighbors

In [ ]:
!pip install geopandas --upgrade

In [ ]:
filtered_dongs = incheon_boundary[incheon_boundary['name_eng'].str.endswith('-dong')]['name_eng']
filtered_dongs.to_csv('/content/drive/MyDrive/filtered_dongs.csv', index=False)

In [ ]:
import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 14
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 제약조건 설정 부분
# ----------------------------------------------
# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(num_i):
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 그 값이 200과 750 사이에 있도록 제약 조건을 설정합니다.
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j >= 200 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

# l_ij에 대한 제약 조건 추가
for i in range(num_i):
    for j in range(num_j):
        # 해당 수요지와 시설물 입지점의 동명을 가져옴
        dong_name_data = data['동명'].iloc[i]
        dong_name_target = target['동명'].iloc[j]

        # 동명이 같거나 인접한 동일 경우에만 할당 가능하도록 함
        if dong_name_data != dong_name_target and dong_name_target not in adjacent_dongs[dong_name_data]:
            prob += l[i][j] == 0

import pulp
from pulp import LpConstraint

# 데이터 초기화 부분
# ----------------------------------------------
# i의 개수, 즉 수요지의 개수
num_i = data.shape[0]

# j의 개수, 즉 시설물 입지점의 개수
num_j = target.shape[0]

# a_i 값의 리스트. 각 수요지 i의 수요량을 나타냅니다.
a = []
for i in data['초등학령인구']:
  a.append(i)

# b_ij 값의 2차원 리스트. 수요지 i와 시설물 입지점 j 사이의 거리를 나타냅니다.
# 예: b = [[b_11, b_12, ...], [b_21, b_22, ...], ...]
b = []

# 각 수요지에서 모든 시설물까지의 거리를 계산
for i, data_row in data.iterrows():
    distances_for_this_demand = []
    for j, target_row in target.iterrows():
        distance = haversine_distance(data_row['Y'], data_row['X'], target_row['Y'], target_row['X'])
        distances_for_this_demand.append(distance)
    b.append(distances_for_this_demand)

# 설치할 시설물의 개수
# 14개 이상 50개 이하
p = 16
# ----------------------------------------------

# LP 문제 정의. 여기서는 총 거리를 최소화하는 것이 목적입니다.
prob = pulp.LpProblem("Binary_Optimization", pulp.LpMinimize)

# 변수 정의 부분
# ----------------------------------------------
# l_ij는 노드 j의 시설물이 노드 i의 총 수요를 충족하면 1, 아니면 0입니다.
l = pulp.LpVariable.dicts("l", (range(num_i), range(num_j)), 0, 1, pulp.LpBinary)

# k_j는 노드 j에 시설물이 설치되면 1, 아니면 0입니다.
k = pulp.LpVariable.dicts("k", range(num_j), 0, 1, pulp.LpBinary)
# ----------------------------------------------

# 목적함수 정의. 각 수요지와 시설물 입지점 사이의 거리와 해당 수요지의 수요량을 곱한 값을 최소화합니다.
prob += pulp.lpSum([a[i] * b[i][j] * l[i][j] for i in range(num_i) for j in range(num_j)])

# 제약조건 설정 부분
# ----------------------------------------------
# 각 수요지 i는 정확히 하나의 시설물 j에만 할당되어야 합니다.
for i in range(num_i):
    prob += pulp.lpSum([l[i][j] for j in range(num_j)]) == 1

# 설치된 시설물의 총 개수는 p와 동일해야 합니다.
prob += pulp.lpSum([k[j] for j in range(num_j)]) == p

# l_ij는 k_j에 의존적입니다. 즉, k_j가 1일 때만 l_ij는 1이 될 수 있습니다.
for i in range(num_i):
    for j in range(num_j):
        prob += l[i][j] <= k[j]

# 각 시설물 j에 할당된 총 수요량을 계산하고, 그 값이 200과 750 사이에 있도록 제약 조건을 설정합니다.
for j in range(num_j):
    total_demand_for_j = pulp.lpSum([a[i] * l[i][j] for i in range(num_i)])
    prob += total_demand_for_j >= 100 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.
    prob += total_demand_for_j <= 750 * k[j]  # 시설물이 설치되었을 때만 제약이 활성화되어야 합니다.

DISTANCE_THRESHOLDS = [1.5, 2, 2.5, 3]

for i in range(num_i):
    possible_assignments = []

    # 거리 기준별로 가능한 시설물 할당 리스트 구하기
    for threshold in DISTANCE_THRESHOLDS:
        possible_assignments = [j for j in range(num_j) if b[i][j] <= threshold]

        if possible_assignments:
            break

    if not possible_assignments:
        # 어떠한 시설물 j와도 연결할 수 없는 경우 (이론적으로 발생하지 않아야 함)
        closest_facility = min(range(num_j), key=lambda j: b[i][j])
        prob += l[i][closest_facility] == 1
    else:
        # 가능한 시설물 외에는 연결을 금지 (0으로 설정)
        for j in range(num_j):
            if j not in possible_assignments:
                prob += l[i][j] == 0

# ----------------------------------------------

# 최적화 문제 풀기
prob.solve()

# 결과 출력 부분
# ----------------------------------------------
print(f"Status: {pulp.LpStatus[prob.status]}")

# 각 수요지에 할당된 시설물 입지점 출력 (값이 1인 경우만 출력)
for i in range(num_i):
    for j in range(num_j):
        if l[i][j].varValue == 1:
            print(f"수요지 {i+1}는 시설물 입지점 {target['학교 id'].iloc[j]}에 할당됨")

# 설치된 시설물 입지점만 출력 (값이 1인 경우만 출력)
for j in range(num_j):
    if k[j].varValue == 1:
        print(f"시설물 입지점 {target['학교 id'].iloc[j]}에 설치됨")

# 할당된 시설물 입지점 및 연결된 수요지 정보를 저장할 리스트
assigned_data = []

# 설치된 시설물 입지점 및 배정된 수요량 정보를 저장할 리스트
facility_demand_data = []

# 각 수요지에 할당된 시설물 입지점 정보 추출
for j in range(num_j):
    if k[j].varValue == 1:
        connected_demands = [i+1 for i in range(num_i) if l[i][j].varValue == 1]
        assigned_data.append({'시설물_입지점': target['학교 id'].iloc[j], '연결된_수요지': connected_demands})

        total_demand_for_j = sum([a[i-1] for i in connected_demands]) # <- 수정된 부분
        facility_demand_data.append({'시설물_입지점': target['학교 id'].iloc[j], '배정된_수요량': total_demand_for_j})

# 데이터프레임으로 변환 후 출력
assigned_df = pd.DataFrame(assigned_data)
print(assigned_df)

facility_demand_df = pd.DataFrame(facility_demand_data)
print(facility_demand_df)